In [96]:
import json 
import csv 
import pandas
import pandas as pd

In [97]:
data = pd.read_csv('Desktop/Sarcasm_Corr/Twitter_Training2Contexts.csv')
pd.options.display.max_columns = None
df = data[['context/0','response','label']].copy()
df.head()

,context/0,response,label
0,@USER If your child isn't named Barron ... #Be...,@USER @USER @USER I don't get this .. obviousl...,SARCASM
1,@USER @USER having to make up excuses of why y...,@USER @USER trying to protest about . Talking ...,SARCASM
2,@USER I ’ ll remember to not support you at th...,@USER @USER @USER He makes an insane about of ...,SARCASM
3,@USER But not half as stupid as Schiff looks ....,@USER @USER Meanwhile Trump won't even release...,SARCASM
4,@USER They already did . Obama said many times...,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,SARCASM


In [98]:
"""### Vader """
def vader(sentence):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    vader = SentimentIntensityAnalyzer()
    score = vader.polarity_scores(sentence)
    return score

# creating lists to keep pos, neu, neg, and compound scores --- later to be used to create a dataframe


# extracting vader scores for each entry in the data
# (we're not using context yet.)
# note that the compound score is rescaled to the [0,1] range
# some classiifers don't take negative values (e.g., MultinomialNB)


def vader_scores(df):
    vs_compound = []
    vs_pos = []
    vs_neu = []
    vs_neg = []
    for row in df:
        score = vader(row)
        neg = float(score['neg'])
        vs_neg.append(neg)
        neu =float(score['neu'])
        vs_neu.append(neu)
        pos =float(score['pos'])
        vs_pos.append(pos)
        compound = float((score['compound']+1)/2) # rescaling to the [0,1] range
        vs_compound.append(compound)
    return vs_compound, vs_pos, vs_neu, vs_neg


In [99]:
cvs_compound, cvs_pos, cvs_neu, cvs_neg = vader_scores(df['context/0'])  #compute vader scores for context/0

df = df.assign(cntx_vader_neu = cvs_neu, cntx_vader_pos = cvs_pos, cntx_vader_neg = cvs_neg, cntx_vader_compound = cvs_compound)

In [100]:
rvs_compound, rvs_pos, rvs_neu, rvs_neg = vader_scores(df['response'])  #compute vader scores for response
df = df.assign(resp_vader_neu = rvs_neu, resp_vader_pos = rvs_pos, resp_vader_neg = rvs_neg, resp_vader_compound = rvs_compound)


In [101]:
df1 = df[df["label"]=="SARCASM"].copy()  #Divide DATA by label 
df2 = df[df["label"]=="NOT_SARCASM"].copy() #Divide DATA by label

# Compare Response to Context

## VADER

In [103]:
from scipy import stats

def correlation_calc(response, context, a):   #Pearson
    r_list = response.tolist()
    c_list = context.tolist()
    correlation,p_value = stats.pearsonr(r_list,c_list)
    return a, correlation

def correlation_calcSpearman(response, context, a):   #Spearman
    r_list = response.tolist()
    c_list = context.tolist()
    correlation,p_value = stats.spearmanr(r_list,c_list)
    return a, correlation


Compute the Correlation of the Vader Compound Scores for Sarcastic Responses and the Sarcastic Context that preceeded them. Compute the correlation of the Non-Sarcastic Responses to their corresponding contexts. 
<br>
Continue doing this for all Vader Values (Negative, Positive, Neutral, and Compound)

In [104]:
print('Pearson')
print('Compound')
print(correlation_calc(df1['resp_vader_compound'], df1['cntx_vader_compound'], "Sarcasm Vader Compound Correlation: "))
print(correlation_calc(df2['resp_vader_compound'], df2['cntx_vader_compound'], "Not Sarcasm Vader Compound Correlation: "))
print()
print('Positive')
print(correlation_calc(df1['resp_vader_pos'], df1['cntx_vader_pos'], "Sarcasm Vader Positive Correlation: "))
print(correlation_calc(df2['resp_vader_pos'], df2['cntx_vader_pos'], "Not Sarcasm Vader Positive Correlation: "))
print()
print('Negative')
print(correlation_calc(df1['resp_vader_neg'], df1['cntx_vader_neg'], "Sarcasm Vader Negative Correlation: "))
print(correlation_calc(df2['resp_vader_neg'], df2['cntx_vader_neg'], "Not Sarcasm Vader Negative Correlation: "))
print()
print('Neutral')
print(correlation_calc(df1['resp_vader_neu'], df1['cntx_vader_neu'], "Sarcasm Vader Neutral Correlation: "))
print(correlation_calc(df2['resp_vader_neu'], df2['cntx_vader_neu'], "Not Sarcasm Vader Neutral Correlation: "))


print()
print()

print('Spearman')
print('Compound')
print(correlation_calcSpearman(df1['resp_vader_compound'], df1['cntx_vader_compound'], "Sarcasm Vader Compound Correlation: "))
print(correlation_calcSpearman(df2['resp_vader_compound'], df2['cntx_vader_compound'], "Not Sarcasm Vader Compound Correlation: "))
print()
print('Positive')
print(correlation_calcSpearman(df1['resp_vader_pos'], df1['cntx_vader_pos'], "Sarcasm Vader Positive Correlation: "))
print(correlation_calcSpearman(df2['resp_vader_pos'], df2['cntx_vader_pos'], "Not Sarcasm Vader Positive Correlation: "))
print()
print('Negative')
print(correlation_calcSpearman(df1['resp_vader_neg'], df1['cntx_vader_neg'], "Sarcasm Vader Negative Correlation: "))
print(correlation_calcSpearman(df2['resp_vader_neg'], df2['cntx_vader_neg'], "Not Sarcasm Vader Negative Correlation: "))
print()
print('Neutral')
print(correlation_calcSpearman(df1['resp_vader_neu'], df1['cntx_vader_neu'], "Sarcasm Vader Neutral Correlation: "))
print(correlation_calcSpearman(df2['resp_vader_neu'], df2['cntx_vader_neu'], "Not Sarcasm Vader Neutral Correlation: "))


Pearson
Compound
('Sarcasm Vader Compound Correlation: ', 0.07605447332942038)
('Not Sarcasm Vader Compound Correlation: ', 0.2749945641751499)

Positive
('Sarcasm Vader Positive Correlation: ', 0.0872712783097193)
('Not Sarcasm Vader Positive Correlation: ', 0.28833031334280174)

Negative
('Sarcasm Vader Negative Correlation: ', 0.13894980844486815)
('Not Sarcasm Vader Negative Correlation: ', 0.222379548228165)

Neutral
('Sarcasm Vader Neutral Correlation: ', 0.15577725670092682)
('Not Sarcasm Vader Neutral Correlation: ', 0.2080424030509918)


Spearman
Compound
('Sarcasm Vader Compound Correlation: ', 0.07210957829895888)
('Not Sarcasm Vader Compound Correlation: ', 0.30928573118032054)

Positive
('Sarcasm Vader Positive Correlation: ', 0.0689342310373311)
('Not Sarcasm Vader Positive Correlation: ', 0.25313823098823024)

Negative
('Sarcasm Vader Negative Correlation: ', 0.11009089507947474)
('Not Sarcasm Vader Negative Correlation: ', 0.23755676598443232)

Neutral
('Sarcasm Vader N

# VAD

Valence/Arousal/Dominance

Using Warriner et al.'s dictionary for norms of valence, arousal, and dominance (VAD) for 13,915 English lemmas, we compute the average VAD scores for each tweet. 

Next, we compute the correlation between the valence/arousal/dominance score of the sarcastic responses and their corresponding preceeding contexts, and compare this to the correlation between the correlation of the VAD score of the non-sarcastic responses & contexts. We hypothesize that the Not-Sarcastic responses will have a higher correlation with their preceeding context since Sarcasm can be marked by a change in emotion.


In [105]:

# Vad will pull all the words that are found in the dictionary, then sum up all the scores, and divide by # of words that matched dictionary entry
import pandas as pd
import nltk



"""### Dataframe
Columns: Context & Response
"""

df.loc[:,'tokenized_context']  = df.loc[:,'context/0'].str.replace('[^\w\s]','')  #get rid of punctuation
df['tokenized_response']  = df.loc[:,'response'].str.replace('[^\w\s]','')  #get rid of punctuation

"""tokenize context & response"""
df.loc[:,'tokenized_context'] = df.apply(lambda row: nltk.word_tokenize(row['tokenized_context']), axis=1) #tokenize
df.loc[:,'tokenized_response'] = df.apply(lambda row: nltk.word_tokenize(row['tokenized_response']), axis=1) #tokenize



from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

class LemmatizationWithPOSTagger(object):
    def __init__(self):
        pass
    def get_wordnet_pos(self,treebank_tag):
        """
        return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
        """
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            # As default pos in lemmatization is Noun
            return wordnet.NOUN

    def pos_tag(self,tokens):
        # find the pos tagginf for each tokens [('What', 'WP'), ('can', 'MD'), ('I', 'PRP') ....
        pos_tokens = [nltk.pos_tag(token) for token in tokens]

        # lemmatization using pos tagg   
        # convert into feature set of [('What', 'What', ['WP']), ('can', 'can', ['MD']), ... ie [original WORD, Lemmatized word, POS tag]
        pos_tokens = [ [(lemmatizer.lemmatize(word,self.get_wordnet_pos(pos_tag))) for (word,pos_tag) in pos] for pos in pos_tokens]
        #pos_tokens = [ [(lemmatizer.lemmatize(word,self.get_wordnet_pos(pos_tag)), [pos_tag]) for (word,pos_tag) in pos] for pos in pos_tokens]

        return pos_tokens

lemmatizer = WordNetLemmatizer()
lemmatization_using_pos_tagger = LemmatizationWithPOSTagger()

#step 2 lemmatization

df.loc[:,'lemma_response'] = lemmatization_using_pos_tagger.pos_tag((df.loc[:,'tokenized_response'])) #response lemma
df.loc[:,'lemma_context'] = lemmatization_using_pos_tagger.pos_tag((df.loc[:,'tokenized_context'])) #context lemma


"""VAD dictionary"""
import csv
reader = csv.reader(open('Desktop/UPDATED_NLP_COURSE/vad1.csv'))

d = {}
for row in reader:
    key = row[1]
    value = row[2:]
    d[key] = value
    
d.pop('Word')          


""" CHECK LEMMAS AGAINST VAD DICTIONARY"""

"""response"""

list_totals = [[(list(map(float,d[token]))) for token in row if token in d] for row in df['lemma_response'].array]

def find_sum(x):
    total = [[num[x] for num in l]for l in list_totals]
    full_list = []
    for i in total:
        if len(i) != 0:
            full_list.append(i)
        else:
            i = [5.0,5.0,5.0]    #if there is a tweet that has nothing, assign average value
            full_list.append(i)
    average = [((sum(x))/(len(x))) for x in full_list]     #sum of all values divided by how many there were
    return average


"""Response"""
vr_total = find_sum(0)
ar_total = find_sum(1)
dr_total = find_sum(2)

df.loc[:,'valence_response'] = vr_total
df.loc[:,'arousal_response'] = ar_total
df.loc[:,'dominance_response'] = dr_total



"""Context"""

list_totals = [[(list(map(float,d[token]))) for token in row if token in d] for row in df['lemma_context'].array]


vc_total = find_sum(0)
ac_total = find_sum(1)
dc_total = find_sum(2)


df.loc[:,'valence_context'] = vc_total
df.loc[:,'arousal_context'] = ac_total
df.loc[:,'dominance_context'] = dc_total

In [106]:
# # Divide data by label again 
df3 = df[df["label"]=="SARCASM"].copy()  #Divide DATA by label
df4 = df[df["label"]=="NOT_SARCASM"].copy() #Divide DATA by label

In [107]:
print("Valence")
print(correlation_calc(df3['valence_response'], df3['valence_context'], "Sarcasm Valence Correlation"))
print(correlation_calc(df4['valence_response'], df4['valence_context'], "Not Sarcasm Valence Correlation"))
print()
print("Dominance")
print(correlation_calc(df3['dominance_response'], df3['dominance_context'], "Sarcasm Dominance Correlation"))
print(correlation_calc(df4['dominance_response'], df4['dominance_context'], "Not Sarcasm Dominance Correlation"))
print()
print("Arousal")
print(correlation_calc(df3['arousal_response'], df3['arousal_context'], "Sarcasm Arousal Correlation"))
print(correlation_calc(df4['arousal_response'], df4['arousal_context'], "Not Sarcasm Arousal Correlation"))

Valence
('Sarcasm Valence Correlation', 0.14586568593344873)
('Not Sarcasm Valence Correlation', 0.3009481608205475)

Dominance
('Sarcasm Dominance Correlation', 0.1341486111329201)
('Not Sarcasm Dominance Correlation', 0.20535444999434416)

Arousal
('Sarcasm Arousal Correlation', 0.12180082280460452)
('Not Sarcasm Arousal Correlation', 0.13527132448249019)


# LIWC

In [165]:
import csv 
from liwc import Liwc
df5 = data[['context/0','response','label']].copy()

df5.loc[:,'tokenized_context']  = df5.loc[:,'context/0'].str.replace('[^\w\s]','')  #get rid of punctuation
df5.loc[:,'tokenized_response']  = df5.loc[:,'response'].str.replace('[^\w\s]','')  #get rid of punctuation


"""tokenize context & response"""
df5.loc[:,'tokenized_context'] = df5.apply(lambda row: nltk.word_tokenize(row['tokenized_context']), axis=1) #tokenize
df5.loc[:,'tokenized_response'] = df5.apply(lambda row: nltk.word_tokenize(row['tokenized_response']), axis=1) #tokenize


lwc = Liwc("Desktop/UPDATED_NLP_COURSE/liwc_dictionaries_shared/LIWC2007_English100131.dic") #liwc dictionary


def liwc_parse(data):    #count which categories appear in each tweet & how many times
    liwcresults =[]
    for token in data:  #response scores
        results = (lwc.parse(token))
        liwcresults.append(results)
    return liwcresults

liwc_results = liwc_parse(df5.loc[:,'tokenized_response'])

def liwc_unique_keys(data):   #collect all field names
    liwc_keys = []
    for key_val in range(len(data)):
        for key in data[key_val].keys():
            if key not in liwc_keys:
                liwc_keys.append(key)
    return liwc_keys

liwc_keys = liwc_unique_keys(liwc_results)


#look at each category in range 3,000, if the value is not in the list of keys, append new key. 

with open('liwc_feature_chart_response.csv', 'w') as csv_file:  
    dict_writer = csv.DictWriter(csv_file, liwc_keys)     #liwc keys will be column heasders   
    writer = csv.writer(csv_file)
    dict_writer.writeheader()
    dict_writer.writerows(liwc_results)      #liwc results will be the rows
    
liwcdata = pd.read_csv("liwc_feature_chart_response.csv")   #dataframe of liwc feeatures
df5 = pd.concat([liwcdata, df5], axis=1, sort=False)   #combine dataframes
df5.fillna(0, inplace=True)  #put zeroes wherever there are no values


In [166]:
keep_same = {'context/0', 'response', 'label', 'tokenized_context', 'tokenized_response','time_resp',
 'relativ_resp',
 'funct_resp',
 'pronoun_resp',
 'ipron_resp',
 'adverb_resp',
 'cogmech_resp',
 'excl_resp',
 'leisure_resp',
 'conj_resp',
 'incl_resp',
 'verb_resp',
 'past_resp',
 'social_resp',
 'ppron_resp',
 'i_resp',
 'cause_resp',
 'humans_resp',
 'certain_resp',
 'achieve_resp',
 'preps_resp',
 'tentat_resp',
 'space_resp',
 'affect_resp',
 'filler_resp',
 'posemo_resp',
 'present_resp',
 'they_resp',
 'shehe_resp',
 'negemo_resp',
 'anger_resp',
 'quant_resp',
 'auxverb_resp',
 'article_resp',
 'insight_resp',
 'work_resp',
 'you_resp',
 'motion_resp',
 'discrep_resp',
 'assent_resp',
 'negate_resp',
 'inhib_resp',
 'home_resp',
 'percept_resp',
 'hear_resp',
 'anx_resp',
 'sad_resp',
 'see_resp',
 'money_resp',
 'bio_resp',
 'health_resp',
 'sexual_resp',
 'nonfl_resp',
 'future_resp',
 'swear_resp',
 'ingest_resp',
 'feel_resp',
 'number_resp',
 'body_resp',
 'relig_resp',
 'family_resp',
 'we_resp',
 'death_resp',
 'friend_resp'}
df5.columns = ['{}{}'.format(c, '' if c in keep_same else '_cntx') for c in df5.columns] #add cntx prefix to determine which features belong to contexg

In [169]:
df5 = df5.drop(['context/0', 'response','tokenized_context','tokenized_response'], axis=1)

In [170]:
df6 = df5[df5["label"]=="SARCASM"].copy()  #Divide DATA by label
df7 = df5[df5["label"]=="NOT_SARCASM"].copy() #Divide DATA by label

In [172]:
j =df1.iloc[:, :64] #Sarcasm Context
h =df1.iloc[:,64 :-1] #Sarcasm Response

In [174]:
j

,cntx_vader_compound_resp_cntx_cntx,cntx_vader_neg_resp_cntx_cntx,cntx_vader_neu_resp_cntx_cntx,cntx_vader_pos_resp_cntx_cntx,context/0,label,resp_vader_compound_resp_cntx_cntx,resp_vader_neg_resp_cntx_cntx,resp_vader_neu_resp_cntx_cntx,resp_vader_pos_resp_cntx_cntx,response
0,0.30625,0.168,0.832,0.000,@USER If your child isn't named Barron ... #Be...,SARCASM,0.87530,0.000,0.796,0.204,@USER @USER @USER I don't get this .. obviousl...
1,0.50000,0.000,1.000,0.000,@USER @USER having to make up excuses of why y...,SARCASM,0.11985,0.256,0.744,0.000,@USER @USER trying to protest about . Talking ...
2,0.34555,0.184,0.816,0.000,@USER I ’ ll remember to not support you at th...,SARCASM,0.27130,0.176,0.824,0.000,@USER @USER @USER He makes an insane about of ...
3,0.92570,0.039,0.708,0.253,@USER But not half as stupid as Schiff looks ....,SARCASM,0.24470,0.130,0.870,0.000,@USER @USER Meanwhile Trump won't even release...
4,0.39885,0.100,0.841,0.058,@USER They already did . Obama said many times...,SARCASM,0.83525,0.000,0.807,0.193,@USER @USER Pretty Sure the Anti-Lincoln Crowd...
...,...,...,...,...,...,...,...,...,...,...,...
2495,0.50000,0.000,1.000,0.000,@USER @USER @USER I misspoke . Have read up on...,SARCASM,0.36915,0.108,0.892,0.000,"She's dangerous & self serving , but as an Ame..."
2496,0.26165,0.279,0.721,0.000,@USER @USER Shame on u for justifying the poli...,SARCASM,0.51985,0.172,0.645,0.183,"@USER @USER I can't even say "" shame on you "" ..."
2497,0.36225,0.081,0.919,0.000,@USER @USER You're = you are . Your = belongs ...,SARCASM,0.22885,0.175,0.743,0.082,@USER @USER @USER So if someone was to murder ...
2498,0.50000,0.000,1.000,0.000,"#Russia , #Pakistan & #China to hold 3 - way c...",SARCASM,0.07595,0.353,0.647,0.000,This trilateral consultations aim to form a re...


In [173]:
h = h.reindex(sorted(h.columns), axis=1)
j = j.reindex(sorted(j.columns), axis=1)

In [204]:
#Sarcasm

Sarcasm_Scores_List = []
for i in range(len(h.columns)):
    r_list = h.iloc[:,i].tolist()
    c_list = j.iloc[:,i].tolist()
    correlation, p_value = stats.pearsonr(r_list,c_list)
    print((h.columns[i],j.columns[i],correlation))
    Sarcasm_Scores_List.append((correlation))

('achieve_resp', 'achieve_cntx', 0.08596522186430626)
('adverb_resp', 'adverb_cntx', 0.08531259704593028)
('affect_resp', 'affect_cntx', 0.08709781440329778)
('anger_resp', 'anger_cntx', 0.10625037751469493)
('anx_resp', 'anx_cntx', 0.041087704286221806)
('article_resp', 'article_cntx', 0.05632156887423635)
('assent_resp', 'assent_cntx', -0.038399515854432124)
('auxverb_resp', 'auxverb_cntx', 0.16595676987654373)
('bio_resp', 'bio_cntx', 0.09700002330000784)
('body_resp', 'body_cntx', 0.06101402152986163)
('cause_resp', 'cause_cntx', 0.008483045490866755)
('certain_resp', 'certain_cntx', -0.0009506283670423291)
('cogmech_resp', 'cogmech_cntx', 0.04978660740880248)
('conj_resp', 'conj_cntx', -0.014211437486800878)
('death_resp', 'death_cntx', 0.09729936234894491)
('discrep_resp', 'discrep_cntx', 0.024179798420093566)
('excl_resp', 'excl_cntx', -0.013357967814571894)
('family_resp', 'family_cntx', 0.05039095683494532)
('feel_resp', 'feel_cntx', 0.030187006337216896)
('filler_resp', 'fill

In [205]:
k =df2.iloc[:, :64] #NotSarcasm context
m =df2.iloc[:,64 :-1] #NotSarcasm response


In [219]:
#Not Sarcasm
k = k.reindex(sorted(k.columns), axis=1)
m = m.reindex(sorted(m.columns), axis=1)

for i in range(len(m.columns)):
    r_list = m.iloc[:,i].tolist()
    c_list = k.iloc[:,i].tolist()
    correlation, p_value = stats.pearsonr(r_list,c_list)
    print(m.columns[i],k.columns[i],correlation)
    Not_Sarcasm_Scores_List.append((correlation))

achieve_resp achieve_cntx 0.07145373930128096
adverb_resp adverb_cntx 0.0520418147155224
affect_resp affect_cntx 0.08734424883723413
anger_resp anger_cntx 0.07277417162018122
anx_resp anx_cntx 0.07586829248759333
article_resp article_cntx 0.08514027692952233
assent_resp assent_cntx 0.04161970206990138
auxverb_resp auxverb_cntx 0.09013935118686361
bio_resp bio_cntx 0.1467246416716215
body_resp body_cntx 0.10095073400456936
cause_resp cause_cntx 0.03910743840639916
certain_resp certain_cntx 0.0509487056494122
cogmech_resp cogmech_cntx 0.12885291122826198
conj_resp conj_cntx 0.08840081091221613
death_resp death_cntx 0.06737583210186694
discrep_resp discrep_cntx 0.00934146220533316
excl_resp excl_cntx 0.0807508010721035
family_resp family_cntx 0.038098585022822234
feel_resp feel_cntx 0.014433955821640906
filler_resp filler_cntx 0.01982373351227297
friend_resp friend_cntx 0.08091835183769447
funct_resp funct_cntx 0.09686464576550413
future_resp future_cntx 0.014296753952200284
health_resp h

In [220]:
def Average(lst): 
    return sum(lst) / len(lst)

print("NotSarcasm Average", (Average(Not_Sarcasm_Scores_List)))
print()
print("Sarcasm Average", (Average(Sarcasm_Scores_List)))



NotSarcasm Average 0.06027429415156402

Sarcasm Average 0.06149014526623917


In [ ]:
def correlation_calc(response, context, a):   #Pearson
    r_list = response.tolist()
    c_list = context.tolist()
    correlation,p_value = stats.pearsonr(r_list,c_list)
    
    